# TP – Analyse de la qualité de l'air
## Etape 3 – Nettoyage avancé Pandas

### Imports

In [1]:
import pandas as pd
import numpy as np
from dateutil import parser

### Charger `weather_raw.csv` avec Pandas:

In [2]:
data_file_path = "../data/weather_raw.csv"
#df_weather = pd.read_csv(data_file_path, dtype={"timestamp": str})
df_weather = pd.read_csv(data_file_path)
df_weather.head()

,city,timestamp,temperature_c,humidity_pct,wind_speed_kmh,precipitation_mm,weather_condition
0,Grenoble,2024-03-12 09:00:00,14.8,74.6,24.8,2.4,pluvieux
1,Strasbourg,05/04/2024 21:00,10.8,78.4,5.7,0.0,neigeux
2,Marseille,28/01/2024 23:00,13.3,60.4,45.7,3.5,pluvieux
3,Bordeaux,29/01/2024 12:00,1.3,84.1,43.3,0.0,brumeux
4,Marseille,2024-01-15 00:00:00,13.3,87.4,21.7,6.0,orageux


### Schema de la df_weather:

In [3]:
df_weather.info()

<class 'pandas.DataFrame'>
RangeIndex: 42172 entries, 0 to 42171
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   city               42172 non-null  str    
 1   timestamp          42172 non-null  str    
 2   temperature_c      41724 non-null  str    
 3   humidity_pct       41760 non-null  float64
 4   wind_speed_kmh     42172 non-null  float64
 5   precipitation_mm   42172 non-null  float64
 6   weather_condition  41310 non-null  str    
dtypes: float64(3), str(4)
memory usage: 2.3 MB


### Identifier et traiter les valeurs manquantes :
  - Interpolation linéaire pour température et humidité:

1) Les colonnes qui ont des NaN :

In [4]:
df_weather.isna().sum()

city                   0
timestamp              0
temperature_c        448
humidity_pct         412
wind_speed_kmh         0
precipitation_mm       0
weather_condition    862
dtype: int64

=> Il y'a des NaN dans **temperature_c**, **humidity_pct** et weather_condition.    

2) L’interpolation linéaire a du sens dans l’ordre temporel: donc on s’assure que timestamp est bien en datetime et on trie:

In [5]:
## == pandas: safe conversion de la colonne timestamp en date time
# Forcer la colonne "timestamp" en str
df_weather["timestamp"] = df_weather["timestamp"].astype(str)

# Strip whitespace
df_weather["timestamp"] = df_weather["timestamp"].str.strip()

# Parse robustly
def parse_timestamp(x):
    if pd.isna(x) or x == "":
        return pd.NaT
    try:
        return parser.parse(x, dayfirst=True)
    except (ValueError, TypeError):
        return pd.NaT

df_weather["timestamp"] = df_weather["timestamp"].apply(parse_timestamp)

## == Check
# print(df_weather[["city", "timestamp"]])
# print(df_weather[df_weather["timestamp"].isna()])
# df_weather.isna().sum()
# df_weather.info()

## == Sort by datetime
df_weather = df_weather.sort_values("timestamp")
df_weather.head()


,city,timestamp,temperature_c,humidity_pct,wind_speed_kmh,precipitation_mm,weather_condition
1212,Nantes,2024-01-01,10.9,77.8,7.2,0.0,neigeux
6149,Bordeaux,2024-01-01,"11,1",64.7,1.0,4.1,pluvieux
37525,Toulouse,2024-01-01,15.2,77.2,17.6,9.7,orageux
8962,Strasbourg,2024-01-01,"5,5",54.3,42.8,0.0,pluvieux
38226,Grenoble,2024-01-01,10.7,77.4,37.4,0.0,NaN


3) S'assurer du type des valeurs de temperature_c et humidity_pct :
D'après le schéma de la df_weather:
temperature_c => est en str => KO
humidity_pct => float64     => OK

Conversion de temperature_c en float:

Les valeurs non convertible en float:

In [6]:
df_weather.loc[
    pd.to_numeric(df_weather["temperature_c"], errors="coerce").isna(),
    "temperature_c"
].unique()[:10]

<StringArray>
['11,1', '5,5', nan, '-1,4', '14,9', '1,7', '0,8', '3,4', '-0,5', '12,8']
Length: 10, dtype: str

Solution pour convertir en float:

In [7]:
if "temperature_c" in df_weather.columns:
    # Convertir en string temporairement pour remplacer la virgule
    if df_weather["temperature_c"].astype(str).str.contains(",").any():
        df_weather["temperature_c"] = (
            df_weather["temperature_c"].astype(str)
            .str.replace(",", ".", regex=False)
            .astype(float)
        )


df_weather.info()

<class 'pandas.DataFrame'>
Index: 42172 entries, 1212 to 26152
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   city               42172 non-null  str           
 1   timestamp          42172 non-null  datetime64[us]
 2   temperature_c      41724 non-null  float64       
 3   humidity_pct       41760 non-null  float64       
 4   wind_speed_kmh     42172 non-null  float64       
 5   precipitation_mm   42172 non-null  float64       
 6   weather_condition  41310 non-null  str           
dtypes: datetime64[us](1), float64(4), str(2)
memory usage: 2.6 MB


3) Interpolation linéaire :
- Température :

In [8]:
df_weather["temperature_c"] = df_weather["temperature_c"].interpolate(method="linear")


- Humidité :

In [9]:
df_weather["humidity_pct"] = df_weather["humidity_pct"].interpolate(method="linear")

Vérifier qu'il n'y a plus de NaN dans les colonnes temperature_c & humidity_pct:

In [10]:
df_weather.isna().sum()

city                   0
timestamp              0
temperature_c          0
humidity_pct           0
wind_speed_kmh         0
precipitation_mm       0
weather_condition    862
dtype: int64

==> Il n'y a plus de NaN dans ces 2 colonnes après l'interpolation.

  - Forward fill pour les conditions météo:
  (on a 862 NaN dans weather_condition)
  forward fill (ffill) : On prend la dernière valeur connue et on la recopie pour remplir les NaN suivants cat on suppose que la condition ne change pas brutalement entre deux mesures :

In [11]:
df_weather["weather_condition"] = df_weather["weather_condition"].ffill()


Vérifier qu'il n'y a plus de NaN dans la colonne weather_condition :

In [12]:
df_weather.isna().sum()

city                 0
timestamp            0
temperature_c        0
humidity_pct         0
wind_speed_kmh       0
precipitation_mm     0
weather_condition    0
dtype: int64

==> Il n'y a plus de NaN dans cette colonne (weather_condition) après Forward fill.

### Corriger les valeurs aberrantes :
#### Températures hors [-40, 50] : remplacer par NaN puis interpoler:

In [13]:
## Remplacer les températures < -40 ou > 50 par NaN :
df_weather.loc[(df_weather["temperature_c"] < -40) | (df_weather["temperature_c"] > 50), "temperature_c"] = np.nan

Test si il y'a de nouveau NaN dans temperature_c :

In [14]:
df_weather["temperature_c"].isna().sum()  # combien de NaN maintenant
df_weather[df_weather["temperature_c"].isna()]    # afficher les lignes concernées

,city,timestamp,temperature_c,humidity_pct,wind_speed_kmh,precipitation_mm,weather_condition
36307,Lille,2024-01-02 00:00:00,NaN,92.3,27.1,6.7,pluvieux
40918,Paris,2024-01-02 03:00:00,NaN,82.2,23.5,0.0,brumeux
20747,Rouen,2024-01-02 18:00:00,NaN,61.5,47.0,0.0,nuageux
8166,Nantes,2024-01-03 10:00:00,NaN,91.4,40.6,0.0,brumeux
18452,Marseille,2024-01-03 12:00:00,NaN,69.1,6.0,0.0,nuageux
...,...,...,...,...,...,...,...
14697,Grenoble,2024-12-03 12:00:00,NaN,82.1,36.6,0.0,nuageux
37142,Strasbourg,2024-12-03 13:00:00,NaN,56.0,18.6,0.0,brumeux
7489,Toulouse,2024-12-04 11:00:00,NaN,52.8,31.3,0.0,nuageux
35757,Toulouse,2024-12-05 18:00:00,NaN,80.2,33.5,0.0,brumeux


Interpolation:

In [15]:
df_weather["temperature_c"] = df_weather["temperature_c"].interpolate(method="linear")

# test:
df_weather["temperature_c"].isna().sum()  # combien de NaN maintenant => 0

np.int64(0)

#### Humidité hors [0, 100] : clipper:

In [16]:
df_weather["humidity_pct"] = df_weather["humidity_pct"].clip(lower=0, upper=100)
df_weather["humidity_pct"].min(), df_weather["humidity_pct"].max() # Vérification


(np.float64(40.0), np.float64(100.0))

### Standardiser les formats de dates:
Déjà fait pour l'intérpolation des valeurs de la colonne temperature_c.

### Fusionner avec les données de pollution (jointure sur ville et heure arrondie):

#### Etapes: 
	- fusionner "air_quality" avec "stations" pour obtenir "city";
	- fusionner "le résultat" avec "weather" sur "city" et "timestamp" arrondie en heure.
	
Les colonnes impliquées dans la fusion: "city", "timestamp".

In [17]:
## Chargement des 2 csv impliqués dans la fusion:
df_air = pd.read_csv("../data/air_quality_raw.csv")
df_stations = pd.read_csv("../data/stations.csv")

## Safe conversion de timestamp de df_air (la colonne timestamp de df_weather est déjà en datetime):
# Forcer la colonne "timestamp" en str
df_air["timestamp"] = df_air["timestamp"].astype(str)

# Strip whitespace
df_air["timestamp"] = df_air["timestamp"].str.strip()

# Parse robustly
def parse_timestamp(x):
    if pd.isna(x) or x == "":
        return pd.NaT
    try:
        return parser.parse(x, dayfirst=True)
    except (ValueError, TypeError):
        return pd.NaT

df_air["timestamp"] = df_air["timestamp"].apply(parse_timestamp)

# == Check
# df_air.isna().sum()
# df_air.info()
# df_air.head(50)

## Arrondir à l’heure pour la jointure:
df_weather["hour"] = df_weather["timestamp"].dt.floor("h") # .floor("H") = arrondi à l’heure inférieure. (ex: 10:42 → 10:00)
df_air["hour"] = df_air["timestamp"].dt.floor("h")
# == Check
# df_weather.head()
# df_air.head()

## Ajouter la ville aux données de pollution
df_air_station = df_air.merge(
    #df_stations[["station_id", "city"]],
    df_stations,
    on="station_id",
    how="left"
)

## Fusion pollution + météo (jointure finale)
#(Jointure sur city + hour)
df_merged = df_air_station.merge(
    df_weather,
    on=["city", "timestamp"],
    how="left",
    suffixes=("_air", "_weather")
)

# df_air_station.head()
df_merged.head()



,station_id,timestamp,pollutant,value,unit,hour_air,station_name,city,lat,lon,station_type,temperature_c,humidity_pct,wind_speed_kmh,precipitation_mm,weather_condition,hour_weather
0,ST0040,2024-07-01 05:00:00,O3,79.29,ug/m3,2024-07-01 05:00:00,Grenoble-urbaine-6,Grenoble,45.160432,5.774254,urbaine,9.9,61.3,7.9,0.0,ensoleille,2024-07-01 05:00:00
1,ST0004,2024-09-06 18:00:00,O3,41.58,ug/m3,2024-09-06 18:00:00,Lyon-urbaine-1,Lyon,45.773049,4.788878,urbaine,NaN,NaN,NaN,NaN,NaN,NaT
2,ST0027,2024-05-23 11:00:00,PM10,29.20,ug/m3,2024-05-23 11:00:00,Nantes-urbaine-1,Nantes,47.184740,-1.565654,urbaine,NaN,NaN,NaN,NaN,NaN,NaT
3,ST0002,2024-03-18 12:00:00,SO2,7.72,ug/m3,2024-03-18 12:00:00,Paris-periurbaine-2,Paris,48.828921,2.375847,periurbaine,NaN,NaN,NaN,NaN,NaN,NaT
4,ST0035,2024-11-06 08:00:00,O3,29.87,ug/m3,2024-11-06 08:00:00,Grenoble-urbaine-1,Grenoble,45.178063,5.765955,urbaine,29.5,50.9,15.6,0.0,nuageux,2024-11-06 08:00:00


### Save to csv (output/pollution_meteo_clean.csv)

In [18]:
df_merged.to_csv("../output/pollution_meteo_clean.csv", index=False)


### Rapport de nettoyage (avant/après par colonne)

#### air_quality_raw.csv
##### Colonnes initiales
station_id, timestamp, pollutant, value, unit
##### Nettoyages appliqués
- Parsing multi-format du timestamp
- Remplacement , → . dans value
- Conversion value en float
- Suppression des doublons
- Gestion des valeurs nulles
##### Tableau avant / après
| Colonne    | Avant                       | Après             |
| ---------- | --------------------------- | ----------------- |
| station_id | string                      | string (inchangé) |
| timestamp  | string, formats hétérogènes | datetime64        |
| pollutant  | string                      | string normalisé  |
| value      | string (`"17,09"`)          | float             |
| unit       | string                      | string (inchangé) |


#### weather_raw.csv 
##### Colonnes initiales
city, timestamp, temperature_c, humidity_pct, wind_speed_kmh, precipitation_mm, weather_condition
##### Nettoyages appliqués
- Conversion de timestamp en datetime
- Arrondi du timestamp à l’heure
- Conversion des colonnes météo en numérique
- Gestion des valeurs manquantes

| Colonne           | Avant (type / problème)   | Après (type / correction) |
| ----------------- | ------------------------- | ------------------------- |
| city              | string                    | string (inchangé)         |
| timestamp         | string, formats multiples | datetime64                |
| temperature_c     | string / float            | float                     |
| humidity_pct      | string / float            | float                     |
| wind_speed_kmh    | string / float            | float                     |
| precipitation_mm  | string / float            | float                     |
| weather_condition | string                    | string (standardisé)      |

#### stations.csv
##### Colonnes initiales
station_id, station_name, city, lat, lon, station_type
=> pas de conversion/nettoyage

#### Nettoyage après fusion des données
##### Opérations
- Jointure air_quality ↔ stations sur station_id
- Jointure pollution ↔ météo sur (city, heure)
- Suppression des lignes sans correspondance météo
- Vérification de cohérence temporelle
##### Colonnes clés finales

| Colonne           | Type final |
| ----------------- | ---------- |
| station_id        | string     |
| station_name      | string     |
| city              | string     |
| timestamp         | datetime64 |
| pollutant         | string     |
| value             | float      |
| temperature_c     | float      |
| humidity_pct      | float      |
| wind_speed_kmh    | float      |
| precipitation_mm  | float      |
| weather_condition | string     |
